<a href="https://colab.research.google.com/github/peterbabulik/QuantumWalker/blob/main/2WalkerFibonacciQW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import time
import os

# --- Parameters ---
N_SITES_1D = 100 # Number of sites in the 1D lattice
DEPTH = 150      # Number of steps in the quantum walk
INITIAL_POS = N_SITES_1D // 2 # Starting position of the walker
INITIAL_COIN = 0              # Initial coin state of the walker (0 or 1)

# --- Coin Matrices for Fibonacci QW ---
# These are the unitary operators applied to the coin state.
# COIN_FIB_0 is used if the Fibonacci sequence element at the walker's site is 0.
# COIN_FIB_1 is used if the Fibonacci sequence element at the walker's site is 1.

H_1Q = (1/np.sqrt(2))*np.array([[1,1],[1,-1]],dtype=np.complex128) # Hadamard coin
X_1Q = np.array([[0,1],[1,0]],dtype=np.complex128)                 # Pauli-X coin

COIN_FIB_0 = H_1Q                  # Example: Hadamard for '0' sites
COIN_FIB_1 = X_1Q @ H_1Q           # Example: X then Hadamard for '1' sites
                                   # Experiment with different coin pairs!
                                   # e.g., COIN_FIB_1 = H_1Q for a standard QW (no Fibonacci effect)
                                   # or COIN_FIB_1 = np.eye(2, dtype=np.complex128) # Identity

# --- Fibonacci Sequence Generation ---
def generate_fibonacci_sequence_01(length):
    """Generates a Fibonacci-type sequence (e.g., S->0, L->1 rule based on S->SL, L->S)
       mapped to 0s and 1s to define the aperiodic lattice.
    """
    if length <= 0:
        return np.array([], dtype=int)
    # Using a common generation: S -> 0, L -> 1. Rules: S_new = S_old + L_old, L_new = S_old
    # Starting with S = "0", L = "1" (for a different flavor than S="0", SL="01")
    # F_0 = "1" (L)
    # F_1 = "0" (S)
    # F_2 = F_1 + F_0 = "01"
    # F_3 = F_2 + F_1 = "010"
    # F_4 = F_3 + F_2 = "01001" etc.
    # Or the one used before: S_n = S_{n-1}S_{n-2} (string concatenation)
    # S_0 = "0", S_1 = "01" -> S_2 = "010", S_3 = "01001"
    s_n_minus_2_str = "0"
    s_n_minus_1_str = "01"
    if length == 1:
        return np.array([int(s_n_minus_2_str[0])], dtype=int) # Return based on S_0

    current_sequence_str = s_n_minus_1_str
    while len(current_sequence_str) < length:
        s_n_str = s_n_minus_1_str + s_n_minus_2_str # String concatenation for Fibonacci words
        current_sequence_str = s_n_str
        s_n_minus_2_str = s_n_minus_1_str
        s_n_minus_1_str = s_n_str

    seq_int = np.array([int(c) for c in current_sequence_str[:length]], dtype=int)
    if len(seq_int) < length:
        seq_int = np.pad(seq_int, (0, length - len(seq_int)), 'wrap')
    return seq_int


# --- Helper Functions for 1D QW State Indexing ---
def get_1d_index(coin_val, site_pos, n_sites_1d):
    if not (0 <= site_pos < n_sites_1d and 0 <= coin_val <= 1):
        raise IndexError(f"Invalid coin ({coin_val}) or pos ({site_pos}) for 1D index (N_sites={n_sites_1d})")
    return coin_val + 2 * site_pos # State vector: [c0p0, c1p0, c0p1, c1p1, ...]

def get_1d_coin_pos_from_index(k, n_sites_1d):
    state_dim_1d = 2 * n_sites_1d
    if not (0 <= k < state_dim_1d):
        raise IndexError(f"Invalid k ({k}) for 1D coin/pos (StateDim={state_dim_1d})")
    coin_val = k % 2
    site_pos = k // 2
    return coin_val, site_pos

# --- Initial State Preparation for Single Walker ---
def prepare_initial_state_1walker(n_sites_1d, p_init, c_init):
    """Prepares the initial state vector for a single walker."""
    state_dim_1d = 2 * n_sites_1d
    initial_state_vector = np.zeros(state_dim_1d, dtype=np.complex128)
    try:
        start_idx = get_1d_index(c_init, p_init, n_sites_1d)
        initial_state_vector[start_idx] = 1.0
        print(f"Initialized Single Walker at (pos={p_init}, coin={c_init}) Index: {start_idx}")
    except IndexError as e:
        print(f"Error during 1-walker initial state prep: {e}")
    return initial_state_vector

# --- Build 1D QW Operators ---
def build_single_qw_coin_matrix_for_site(fib_sequence_site_val, coin_for_0, coin_for_1):
    """Selects the coin operator based on the Fibonacci sequence value at a site."""
    return coin_for_0 if fib_sequence_site_val == 0 else coin_for_1

def build_1walker_coin_operator(n_sites_1d, static_fib_pattern, coin_for_0, coin_for_1):
    """Builds the global coin operator for the 1D QW on the Fibonacci substrate."""
    d_1w = 2 * n_sites_1d # Dimension of the single walker Hilbert space
    U_C = np.zeros((d_1w, d_1w), dtype=np.complex128)
    # For each site, apply the appropriate coin operator only to the coin subspace of that site
    for site_idx in range(n_sites_1d):
        # Determine which coin to use based on the Fibonacci pattern at this site
        coin_op_for_this_site = build_single_qw_coin_matrix_for_site(
            static_fib_pattern[site_idx], coin_for_0, coin_for_1
        )
        # The coin operator acts on the 2x2 coin subspace for 'site_idx'.
        # State vector indices for site_idx are 2*site_idx (coin 0) and 2*site_idx+1 (coin 1)
        U_C[2*site_idx : 2*site_idx+2, 2*site_idx : 2*site_idx+2] = coin_op_for_this_site
    return U_C

def build_single_qw_shift_matrix(n_sites_1d):
    """Builds the global shift operator for the 1D QW."""
    d_1w = 2 * n_sites_1d
    S_1w = np.zeros((d_1w, d_1w), dtype=np.complex128)
    # For each basis state |coin_val, site_pos>, determine where it shifts
    for k_in_1w in range(d_1w):
        coin_val, site_pos = get_1d_coin_pos_from_index(k_in_1w, n_sites_1d)
        # Standard shift: coin 0 moves left, coin 1 moves right
        # Implements periodic boundary conditions
        if coin_val == 0: # Shift left
            new_pos = (site_pos - 1 + n_sites_1d) % n_sites_1d
        else: # coin_val == 1, Shift right
            new_pos = (site_pos + 1) % n_sites_1d

        k_out_1w = get_1d_index(coin_val, new_pos, n_sites_1d)
        S_1w[k_out_1w, k_in_1w] = 1.0 # Amplitude is 1 for the shift
    return S_1w

# --- Observables for Single Walker ---
def calculate_p_pos_1walker(qw_state, n_sites_1d):
    """Calculates the probability distribution over positions."""
    d_1w = 2 * n_sites_1d
    probs_1d = np.zeros(n_sites_1d)
    if len(qw_state) != d_1w: return probs_1d
    for k in range(d_1w):
        _, site_pos = get_1d_coin_pos_from_index(k, n_sites_1d)
        probs_1d[site_pos] += np.abs(qw_state[k])**2
    return probs_1d

def calculate_coin_entanglement_1walker(qw_state, n_sites_1d):
    """Calculates the entanglement entropy of the coin subsystem."""
    # (Implementation from your previous single-walker code)
    d_1w = 2 * n_sites_1d; coin_dim = 2
    if len(qw_state) != d_1w: return np.nan
    norm = np.linalg.norm(qw_state)
    if norm < 1e-9: return 0.0
    current_psi = qw_state / norm if np.abs(norm - 1.0) > 1e-6 else qw_state
    rho_full = np.outer(current_psi, np.conjugate(current_psi))
    rho_coin_reduced = np.zeros((coin_dim, coin_dim), dtype=np.complex128)
    for k_row in range(d_1w):
        c_r, p_r = get_1d_coin_pos_from_index(k_row, n_sites_1d)
        for k_col in range(d_1w):
            c_c, p_c = get_1d_coin_pos_from_index(k_col, n_sites_1d)
            if p_r == p_c: rho_coin_reduced[c_r, c_c] += rho_full[k_row, k_col]
    tr_rho_coin = np.trace(rho_coin_reduced)
    if abs(tr_rho_coin) < 1e-9: return 0.0
    if np.abs(tr_rho_coin - 1.0) > 1e-6 : rho_coin_reduced /= tr_rho_coin
    eigs = np.linalg.eigvalsh(rho_coin_reduced); ent = 0.0; epsilon = 1e-12
    for e_val in eigs: ent -= (e_val * np.log2(e_val) if e_val > epsilon else 0.0)
    return max(0.0, np.real(ent))

# --- Simulation Loop for 1D QW on Static Fibonacci Substrate ---
def run_1d_qw_1walker_fibonacci_substrate(
    n_sites_1d, depth,
    p_init, c_init,
    static_fib_pattern, # The pre-generated Fibonacci sequence
    coin_for_0, coin_for_1 # The two distinct coin operators
):
    current_qw_state = prepare_initial_state_1walker(n_sites_1d, p_init, c_init)

    prob_hist = np.full((depth + 1, n_sites_1d), np.nan)
    ent_hist = np.full(depth + 1, np.nan)
    # Store the static substrate pattern once for plotting convenience
    substrate_pattern_history = [static_fib_pattern.copy()]

    prob_hist[0, :] = calculate_p_pos_1walker(current_qw_state, n_sites_1d)
    ent_hist[0] = calculate_coin_entanglement_1walker(current_qw_state, n_sites_1d)

    print(f"\nStarting 1-Walker 1D QW on static Fibonacci Substrate ({n_sites_1d} sites, {depth} steps)...")
    start_time = time.time()

    # Precompute the shift operator as it's constant
    S_Shift_constant = build_single_qw_shift_matrix(n_sites_1d)
    # Precompute the coin operator as the substrate is static
    U_Coin_constant = build_1walker_coin_operator(n_sites_1d, static_fib_pattern, coin_for_0, coin_for_1)
    # Combine them into the constant step operator
    U_Step_constant = S_Shift_constant @ U_Coin_constant


    for step in range(depth):
        # Apply the constant step operator
        current_qw_state = U_Step_constant @ current_qw_state

        # Normalize state (important for numerical stability)
        norm_qw = np.linalg.norm(current_qw_state)
        if abs(norm_qw) > 1e-9: current_qw_state /= norm_qw
        else: print(f"Warning: QW Norm zero at step {step+1}."); break

        # Calculate observables
        prob_hist[step + 1, :] = calculate_p_pos_1walker(current_qw_state, n_sites_1d)
        ent_hist[step+1] = calculate_coin_entanglement_1walker(current_qw_state, n_sites_1d)

        if substrate_pattern_history is not None and len(substrate_pattern_history) <= depth : # only append if we need to for plotting
             substrate_pattern_history.append(static_fib_pattern.copy())


        if (step + 1) % (max(1,depth // 10)) == 0 or step == depth -1: # Print progress more often
            s_val = ent_hist[step+1]
            print(f"  Fib-QW Step {step + 1}/{depth} completed. S_coin={f'{s_val:.3f}' if np.isfinite(s_val) else 'NaN'}")

    end_time = time.time()
    print(f"1-Walker Fibonacci QW Evolution complete. Time: {end_time - start_time:.2f} seconds.")

    return {
        "prob_hist": prob_hist, "ent_hist": ent_hist,
        "ca_history": np.array(substrate_pattern_history), # Represents the static substrate
        "params": { "n_sites":n_sites_1d,"depth":depth, "substrate_type": "Fibonacci",
                    "p_init":p_init, "c_init":c_init,
                    "coin_0_str": "COIN_FIB_0", "coin_1_str": "COIN_FIB_1" } # Placeholder for coin info
    }

# --- Plotting and Summary (using the slightly adapted version from previous) ---
def plot_1walker_results(results, save_path=None, plot_title_prefix="1-Walker QW"):
    params = results["params"]; p_hist = results["prob_hist"]
    e_hist = results["ent_hist"]; ca_h = results["ca_history"] # This is the static substrate pattern
    n_sites = params["n_sites"]; depth = params["depth"]

    fig, axs = plt.subplots(4, 1, figsize=(12, 15), gridspec_kw={'height_ratios': [1.0, 3, 2, 2]}) # Adjusted CA height

    substrate_info = params.get("substrate_type", "Unknown Substrate")
    title_str = f"{plot_title_prefix} on {substrate_info} (N={n_sites}, D={depth})"
    fig.suptitle(title_str, fontsize=14)

    # Plot the static Fibonacci substrate (just the first instance is enough)
    axs[0].imshow(ca_h[0:1, :], cmap='binary', aspect='auto', interpolation='nearest', extent=[0, n_sites-1, 0, 1])
    axs[0].set_title(f"Static Fibonacci Substrate Pattern (0/1)"); axs[0].set_xlabel("Site Index"); axs[0].set_yticks([])


    time_extent = [0, depth, 0, n_sites-1]
    prob_st=p_hist.T
    # For Fibonacci QW, linear scale often shows the "spiky" fractal structure better
    norm_v = None # Linear scale for probability
    # To use log scale (can sometimes obscure details for spiky fractals):
    # pos_p=prob_st[prob_st>1e-9]; min_v=pos_p.min() if len(pos_p)>0 else 1e-9; max_v=prob_st.max()
    # norm_v=colors.LogNorm(vmin=min_v, vmax=max(max_v, min_v+1e-6) if max_v > min_v else min_v + 1e-6)

    im=axs[1].imshow(prob_st, aspect='auto', origin='lower', cmap='magma', norm=norm_v, extent=time_extent) # Changed cmap
    plt.colorbar(im, ax=axs[1], label="P(x,t)"); axs[1].set_title("Walker P(x,t)"); axs[1].set_xlabel("Time Step"); axs[1].set_ylabel("Site Index (x)")

    ts=np.arange(depth+1)
    axs[2].plot(ts, e_hist, marker='.', ls='-', color='blue', label='Walker Coin Entanglement')
    axs[2].set_title("Coin Entanglement S(t)"); axs[2].set_xlabel("Time Step"); axs[2].set_ylabel("Entropy S (bits)"); axs[2].grid(True); axs[2].legend(); axs[2].set_ylim(bottom=-0.05, top=1.05)

    avg_pos = np.array([np.sum(p_hist[t,:] * np.arange(n_sites)) / np.sum(p_hist[t,:]) if np.sum(p_hist[t,:]) > 1e-9 else np.nan for t in range(depth+1)])
    axs[3].plot(ts, avg_pos, marker='.', ls='-', color='green')
    axs[3].set_title("Average Position <x>(t)"); axs[3].set_xlabel("Time Step"); axs[3].set_ylabel("<x>"); axs[3].grid(True)

    plt.tight_layout(rect=[0,0,1,0.96])

    if save_path:
        plt.savefig(save_path)
        print(f"Plot saved to {save_path}")
        plt.close(fig)
    else:
        plt.show()

# --- Main Execution ---
if __name__ == "__main__":
    output_dir = "qw_fibonacci_substrate_1walker_sim" # New directory name
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created directory: {output_dir}")

    # Generate the static Fibonacci pattern for the substrate
    fibonacci_pattern_for_substrate = generate_fibonacci_sequence_01(N_SITES_1D)
    print(f"Generated Fibonacci Substrate (length {N_SITES_1D}, first 30 elements): {fibonacci_pattern_for_substrate[:min(30, N_SITES_1D)]}")

    print(f"\n===== Running 1-Walker simulation on static Fibonacci Substrate =====")
    print(f"Using COIN_FIB_0 for '0' sites and COIN_FIB_1 for '1' sites.")

    results_fib_qw = run_1d_qw_1walker_fibonacci_substrate(
        n_sites_1d=N_SITES_1D, depth=DEPTH,
        p_init=INITIAL_POS, c_init=INITIAL_COIN,
        static_fib_pattern=fibonacci_pattern_for_substrate,
        coin_for_0=COIN_FIB_0, coin_for_1=COIN_FIB_1 # Pass the actual coin matrices
    )

    plot_filename = os.path.join(output_dir, f"fibonacci_1W_N{N_SITES_1D}_D{DEPTH}.png")
    plot_1walker_results(results_fib_qw, save_path=plot_filename, plot_title_prefix="1D Fibonacci QW")

    print(f"\n<<<<< 1-WALKER FIBONACCI QW SIMULATION COMPLETE >>>>>")
    print(f"Plot saved in ./{output_dir}/")

Created directory: qw_fibonacci_substrate_1walker_sim
Generated Fibonacci Substrate (length 100, first 30 elements): [0 1 0 0 1 0 1 0 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 1 0 1 0 0]

===== Running 1-Walker simulation on static Fibonacci Substrate =====
Using COIN_FIB_0 for '0' sites and COIN_FIB_1 for '1' sites.
Initialized Single Walker at (pos=50, coin=0) Index: 100

Starting 1-Walker 1D QW on static Fibonacci Substrate (100 sites, 150 steps)...
  Fib-QW Step 15/150 completed. S_coin=0.779
  Fib-QW Step 30/150 completed. S_coin=0.915
  Fib-QW Step 45/150 completed. S_coin=0.407
  Fib-QW Step 60/150 completed. S_coin=0.982
  Fib-QW Step 75/150 completed. S_coin=0.844
  Fib-QW Step 90/150 completed. S_coin=0.846
  Fib-QW Step 105/150 completed. S_coin=0.648
  Fib-QW Step 120/150 completed. S_coin=0.696
  Fib-QW Step 135/150 completed. S_coin=0.845
  Fib-QW Step 150/150 completed. S_coin=0.900
1-Walker Fibonacci QW Evolution complete. Time: 9.89 seconds.
Plot saved to qw_fibonacci_substrat

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import time
import os

# --- Parameters (can be overridden in main execution) ---
N_SITES_1D_DEFAULT = 100
DEPTH_DEFAULT = 150
INITIAL_POS_DEFAULT = N_SITES_1D_DEFAULT // 2
# INITIAL_COIN_DEFAULT will be handled more flexibly

# --- Coin Matrix Definitions ---
H_1Q = (1/np.sqrt(2))*np.array([[1,1],[1,-1]],dtype=np.complex128)
X_1Q = np.array([[0,1],[1,0]],dtype=np.complex128)
I_1Q = np.eye(2, dtype=np.complex128) # Identity coin
Z_1Q = np.array([[1,0],[0,-1]], dtype=np.complex128) # Pauli-Z
Y_1Q = np.array([[0,-1j],[1j,0]], dtype=np.complex128) # Pauli-Y

def Rz_gate(phi):
    return np.array([[np.exp(-1j*phi/2), 0],
                     [0, np.exp(1j*phi/2)]], dtype=np.complex128)

# --- Aperiodic Sequence Generation ---
def generate_fibonacci_sequence_01(length):
    s_n_minus_2_str = "0"
    s_n_minus_1_str = "01"
    if length <= 0: return np.array([], dtype=int)
    if length == 1: return np.array([int(s_n_minus_2_str[0])], dtype=int)
    current_sequence_str = s_n_minus_1_str
    while len(current_sequence_str) < length:
        s_n_str = s_n_minus_1_str + s_n_minus_2_str
        current_sequence_str = s_n_str
        s_n_minus_2_str = s_n_minus_1_str
        s_n_minus_1_str = s_n_str
    seq_int = np.array([int(c) for c in current_sequence_str[:length]], dtype=int)
    if len(seq_int) < length: seq_int = np.pad(seq_int, (0, length - len(seq_int)), 'wrap')
    return seq_int

def generate_thue_morse_sequence_01(length):
    """Generates a Thue-Morse sequence of 0s and 1s."""
    if length <= 0: return np.array([], dtype=int)
    tm_seq = [0]
    while len(tm_seq) < length:
        tm_seq.extend([1 - x for x in tm_seq]) # Append the bitwise complement
    return np.array(tm_seq[:length], dtype=int)

# --- Helper Functions for 1D QW State Indexing (Unchanged) ---
def get_1d_index(coin_val, site_pos, n_sites_1d):
    if not (0 <= site_pos < n_sites_1d and 0 <= coin_val <= 1):
        raise IndexError(f"Invalid coin ({coin_val}) or pos ({site_pos}) for 1D index (N_sites={n_sites_1d})")
    return coin_val + 2 * site_pos
def get_1d_coin_pos_from_index(k, n_sites_1d):
    state_dim_1d = 2 * n_sites_1d
    if not (0 <= k < state_dim_1d):
        raise IndexError(f"Invalid k ({k}) for 1D coin/pos (StateDim={state_dim_1d})")
    coin_val = k % 2; site_pos = k // 2
    return coin_val, site_pos

# --- Initial State Preparation (More Flexible) ---
def prepare_initial_state_1walker_flexible(n_sites_1d, p_init, initial_coin_vector):
    """
    Prepares the initial state vector for a single walker.
    initial_coin_vector: A 2-element numpy array representing the coin state (e.g., [1,0] for |0>, [0,1] for |1>, [1/sqrt(2), 1/sqrt(2)] for H|0>).
    """
    state_dim_1d = 2 * n_sites_1d
    initial_state_vector = np.zeros(state_dim_1d, dtype=np.complex128)
    if len(initial_coin_vector) != 2:
        raise ValueError("initial_coin_vector must have 2 elements.")
    if not np.isclose(np.linalg.norm(initial_coin_vector), 1.0):
        print("Warning: Initial coin vector is not normalized. Normalizing...")
        initial_coin_vector = initial_coin_vector / np.linalg.norm(initial_coin_vector)

    try:
        idx_c0 = get_1d_index(0, p_init, n_sites_1d)
        idx_c1 = get_1d_index(1, p_init, n_sites_1d)
        initial_state_vector[idx_c0] = initial_coin_vector[0]
        initial_state_vector[idx_c1] = initial_coin_vector[1]
        print(f"Initialized Single Walker at pos={p_init} with coin_vec=[{initial_coin_vector[0]:.3f}, {initial_coin_vector[1]:.3f}]")
    except IndexError as e:
        print(f"Error during 1-walker initial state prep: {e}")
    return initial_state_vector

# --- Build 1D QW Operators (Unchanged from Fibonacci example) ---
def build_single_qw_coin_matrix_for_site(substrate_site_val, coin_for_0, coin_for_1):
    return coin_for_0 if substrate_site_val == 0 else coin_for_1
def build_1walker_coin_operator(n_sites_1d, static_substrate_pattern, coin_for_0, coin_for_1):
    d_1w = 2 * n_sites_1d
    U_C = np.zeros((d_1w, d_1w), dtype=np.complex128)
    for site_idx in range(n_sites_1d):
        coin_op_for_this_site = build_single_qw_coin_matrix_for_site(
            static_substrate_pattern[site_idx], coin_for_0, coin_for_1)
        U_C[2*site_idx : 2*site_idx+2, 2*site_idx : 2*site_idx+2] = coin_op_for_this_site
    return U_C
def build_single_qw_shift_matrix(n_sites_1d):
    d_1w = 2 * n_sites_1d
    S_1w = np.zeros((d_1w, d_1w), dtype=np.complex128)
    for k_in_1w in range(d_1w):
        coin_val, site_pos = get_1d_coin_pos_from_index(k_in_1w, n_sites_1d)
        if coin_val == 0: new_pos = (site_pos - 1 + n_sites_1d) % n_sites_1d
        else: new_pos = (site_pos + 1) % n_sites_1d
        k_out_1w = get_1d_index(coin_val, new_pos, n_sites_1d)
        S_1w[k_out_1w, k_in_1w] = 1.0
    return S_1w

# --- Observables for Single Walker (Unchanged) ---
def calculate_p_pos_1walker(qw_state, n_sites_1d):
    d_1w = 2 * n_sites_1d; probs_1d = np.zeros(n_sites_1d)
    if len(qw_state) != d_1w: return probs_1d
    for k in range(d_1w):
        _, site_pos = get_1d_coin_pos_from_index(k, n_sites_1d)
        probs_1d[site_pos] += np.abs(qw_state[k])**2
    return probs_1d
def calculate_coin_entanglement_1walker(qw_state, n_sites_1d):
    d_1w = 2 * n_sites_1d; coin_dim = 2
    if len(qw_state) != d_1w: return np.nan
    norm = np.linalg.norm(qw_state)
    if norm < 1e-9: return 0.0
    current_psi = qw_state / norm if np.abs(norm - 1.0) > 1e-6 else qw_state
    rho_full = np.outer(current_psi, np.conjugate(current_psi))
    rho_coin_reduced = np.zeros((coin_dim, coin_dim), dtype=np.complex128)
    for k_row in range(d_1w):
        c_r, p_r = get_1d_coin_pos_from_index(k_row, n_sites_1d)
        for k_col in range(d_1w):
            c_c, p_c = get_1d_coin_pos_from_index(k_col, n_sites_1d)
            if p_r == p_c: rho_coin_reduced[c_r, c_c] += rho_full[k_row, k_col]
    tr_rho_coin = np.trace(rho_coin_reduced)
    if abs(tr_rho_coin) < 1e-9: return 0.0
    if np.abs(tr_rho_coin - 1.0) > 1e-6 : rho_coin_reduced /= tr_rho_coin
    eigs = np.linalg.eigvalsh(rho_coin_reduced); ent = 0.0; epsilon = 1e-12
    for e_val in eigs: ent -= (e_val * np.log2(e_val) if e_val > epsilon else 0.0)
    return max(0.0, np.real(ent))

# --- Simulation Loop for 1D QW on Static Aperiodic Substrate ---
def run_1d_qw_1walker_aperiodic_substrate(
    n_sites_1d, depth,
    p_init, initial_coin_vector,
    static_substrate_pattern,
    coin_for_0, coin_for_1,
    substrate_name="Aperiodic"
):
    current_qw_state = prepare_initial_state_1walker_flexible(n_sites_1d, p_init, initial_coin_vector)
    prob_hist = np.full((depth + 1, n_sites_1d), np.nan)
    ent_hist = np.full(depth + 1, np.nan)
    substrate_pattern_history = [static_substrate_pattern.copy()]
    prob_hist[0, :] = calculate_p_pos_1walker(current_qw_state, n_sites_1d)
    ent_hist[0] = calculate_coin_entanglement_1walker(current_qw_state, n_sites_1d)

    print(f"\nStarting 1-Walker 1D QW on static {substrate_name} Substrate ({n_sites_1d} sites, {depth} steps)...")
    start_time = time.time()
    S_Shift_constant = build_single_qw_shift_matrix(n_sites_1d)
    U_Coin_constant = build_1walker_coin_operator(n_sites_1d, static_substrate_pattern, coin_for_0, coin_for_1)
    U_Step_constant = S_Shift_constant @ U_Coin_constant

    for step in range(depth):
        current_qw_state = U_Step_constant @ current_qw_state
        norm_qw = np.linalg.norm(current_qw_state)
        if abs(norm_qw) > 1e-9: current_qw_state /= norm_qw
        else: print(f"Warning: QW Norm zero at step {step+1}."); break
        prob_hist[step + 1, :] = calculate_p_pos_1walker(current_qw_state, n_sites_1d)
        ent_hist[step+1] = calculate_coin_entanglement_1walker(current_qw_state, n_sites_1d)
        if len(substrate_pattern_history) <= depth : substrate_pattern_history.append(static_substrate_pattern.copy())
        if (step + 1) % (max(1,depth // 10)) == 0 or step == depth -1:
            print(f"  {substrate_name}-QW Step {step + 1}/{depth} completed. S_coin={ent_hist[step+1]:.3f}")
    end_time = time.time()
    print(f"1-Walker {substrate_name} QW Evolution complete. Time: {end_time - start_time:.2f} seconds.")
    return {
        "prob_hist": prob_hist, "ent_hist": ent_hist,
        "ca_history": np.array(substrate_pattern_history),
        "params": { "n_sites":n_sites_1d,"depth":depth, "substrate_type": substrate_name,
                    "p_init":p_init, "initial_coin_vector_str": str(initial_coin_vector),
                    "coin_0_info": "UserDefined", "coin_1_info": "UserDefined" }
    }

# --- Plotting (can reuse from previous, title is more generic) ---
def plot_results(results, save_path=None, plot_title_prefix="1-Walker QW"): # Renamed for clarity
    # (Same plotting function as in the previous Fibonacci example)
    params = results["params"]; p_hist = results["prob_hist"]
    e_hist = results["ent_hist"]; ca_h = results["ca_history"]
    n_sites = params["n_sites"]; depth = params["depth"]
    fig, axs = plt.subplots(4, 1, figsize=(12, 15), gridspec_kw={'height_ratios': [1.0, 3, 2, 2]})
    substrate_info = params.get("substrate_type", "Unknown Substrate")
    title_str = f"{plot_title_prefix} on {substrate_info} (N={n_sites}, D={depth})"
    fig.suptitle(title_str, fontsize=14)
    axs[0].imshow(ca_h[0:1, :], cmap='binary', aspect='auto', interpolation='nearest', extent=[0, n_sites-1, 0, 1])
    axs[0].set_title(f"Static {substrate_info} Substrate Pattern (0/1)"); axs[0].set_xlabel("Site Index"); axs[0].set_yticks([])
    time_extent = [0, depth, 0, n_sites-1]
    prob_st=p_hist.T; norm_v = None # Linear scale for probability
    im=axs[1].imshow(prob_st, aspect='auto', origin='lower', cmap='magma', norm=norm_v, extent=time_extent)
    plt.colorbar(im, ax=axs[1], label="P(x,t)"); axs[1].set_title("Walker P(x,t)"); axs[1].set_xlabel("Time Step"); axs[1].set_ylabel("Site Index (x)")
    ts=np.arange(depth+1)
    axs[2].plot(ts, e_hist, marker='.', ls='-', color='blue', label='Walker Coin Entanglement')
    axs[2].set_title("Coin Entanglement S(t)"); axs[2].set_xlabel("Time Step"); axs[2].set_ylabel("Entropy S (bits)"); axs[2].grid(True); axs[2].legend(); axs[2].set_ylim(bottom=-0.05, top=1.05)
    avg_pos = np.array([np.sum(p_hist[t,:] * np.arange(n_sites)) / np.sum(p_hist[t,:]) if np.sum(p_hist[t,:]) > 1e-9 else np.nan for t in range(depth+1)])
    axs[3].plot(ts, avg_pos, marker='.', ls='-', color='green')
    axs[3].set_title("Average Position <x>(t)"); axs[3].set_xlabel("Time Step"); axs[3].set_ylabel("<x>"); axs[3].grid(True)
    plt.tight_layout(rect=[0,0,1,0.96])
    if save_path: plt.savefig(save_path); print(f"Plot saved to {save_path}"); plt.close(fig)
    else: plt.show()

# --- 4. Quantitative Fractal Analysis (Box-Counting for 1D data) ---
def box_counting_1d(data_array, threshold=1e-5):
    """
    Performs 1D box-counting on a data array (e.g., P(x) at a fixed time).
    Args:
        data_array (np.array): 1D array of values.
        threshold (float): Minimum value to consider a site "occupied".
    Returns:
        tuple: (box_sizes, counts)
    """
    occupied_sites = np.where(data_array > threshold)[0]
    if len(occupied_sites) == 0:
        print("No occupied sites found above threshold for box-counting.")
        return np.array([]), np.array([])

    min_coord = np.min(occupied_sites)
    max_coord = np.max(occupied_sites)
    data_range = max_coord - min_coord + 1
    if data_range <= 0: data_range = 1


    # Define a range of box sizes (powers of 2 often work well, or a logspace)
    # Max box size should not exceed data_range
    max_power = int(np.log2(data_range))
    if max_power < 1: max_power = 1 # Ensure at least one box size
    box_sizes = np.array([2**i for i in range(max_power + 1) if 2**i <= data_range/2 and 2**i > 0]) # ensure s < L/2
    if len(box_sizes) == 0: # Fallback for small data_range
        box_sizes = np.array([1,2,4]) if data_range >=4 else np.array([1])
        box_sizes = box_sizes[box_sizes <= data_range/2]
        if len(box_sizes) == 0 and data_range > 0 : box_sizes = np.array([1])


    counts = []
    for s in box_sizes:
        s = int(s)
        if s == 0: continue
        num_boxes_occupied = 0
        # Iterate through boxes covering the occupied range
        current_pos = min_coord
        while current_pos <= max_coord:
            box_end = current_pos + s -1
            # Check if any occupied site falls within this box
            is_box_occupied = False
            for site in occupied_sites:
                if current_pos <= site <= box_end:
                    is_box_occupied = True
                    break
            if is_box_occupied:
                num_boxes_occupied += 1
            current_pos += s # Move to the next box
        counts.append(num_boxes_occupied)

    return np.array(box_sizes), np.array(counts)

def plot_fractal_dimension_analysis(box_sizes, counts, substrate_name, t_fixed, save_dir):
    if len(box_sizes) < 2 or len(counts) < 2:
        print(f"Not enough data points for fractal dimension plot ({substrate_name}, t={t_fixed}).")
        return

    log_inv_box_sizes = np.log(1.0 / box_sizes) # log(1/s)
    log_counts = np.log(counts)

    plt.figure(figsize=(8, 6))
    plt.plot(log_inv_box_sizes, log_counts, 'o-', label=f'{substrate_name} at t={t_fixed}')

    # Fit a line to estimate fractal dimension (slope)
    # Exclude points where counts might be 1 or very small, as log can be problematic
    valid_indices = np.where(counts > 1)[0] # only fit where N(s) > 1
    if len(valid_indices) > 1 :
        slope, intercept = np.polyfit(log_inv_box_sizes[valid_indices], log_counts[valid_indices], 1)
        plt.plot(log_inv_box_sizes, slope * log_inv_box_sizes + intercept, '--',
                 label=f'Fit (Slope Df ≈ {slope:.3f})')
        print(f"Estimated Fractal Dimension Df for {substrate_name} at t={t_fixed}: {slope:.3f}")
    else:
        print(f"Could not perform linear fit for {substrate_name} at t={t_fixed} (too few points with N(s)>1).")


    plt.xlabel("log(1 / Box Size)")
    plt.ylabel("log(Number of Occupied Boxes)")
    plt.title(f"Fractal Dimension Analysis (Box-Counting)")
    plt.legend()
    plt.grid(True)
    if save_dir:
        plt.savefig(os.path.join(save_dir, f"fractal_dim_{substrate_name}_t{t_fixed}.png"))
        print(f"Fractal dimension plot saved for {substrate_name} at t={t_fixed}.")
        plt.close()
    else:
        plt.show()

# --- Main Execution Script ---
if __name__ == "__main__":
    output_base_dir = "qw_aperiodic_explorations"
    if not os.path.exists(output_base_dir): os.makedirs(output_base_dir)

    # --- Experiment Configuration ---
    sim_params = {
        "n_sites": 150, # Increased for better pattern visibility
        "depth": 200,   # Increased for longer evolution
        "p_init_factor": 0.5, # Initial position as fraction of n_sites
    }

    # --- 1. Vary Coins (Fibonacci Substrate) ---
    print("\n===== 1. Varying Coins on Fibonacci Substrate =====")
    fib_substrate = generate_fibonacci_sequence_01(sim_params["n_sites"])
    initial_coin_ket0 = np.array([1,0], dtype=np.complex128) # |0>_c

    coin_pairs_to_test = [
        {"name": "H_vs_XH",   "c0": H_1Q, "c1": X_1Q @ H_1Q},
        {"name": "H_vs_I",    "c0": H_1Q, "c1": I_1Q},        # Identity as strong scatterer
        {"name": "H_vs_H_Rz", "c0": H_1Q, "c1": Rz_gate(np.pi/2) @ H_1Q}, # Similar but phase shifted
        {"name": "H_vs_H",    "c0": H_1Q, "c1": H_1Q},        # Effectively standard QW, no Fibonacci effect
    ]

    for cp_info in coin_pairs_to_test:
        sim_label = f"Fibonacci_{cp_info['name']}"
        print(f"\n--- Running: {sim_label} ---")
        results = run_1d_qw_1walker_aperiodic_substrate(
            n_sites_1d=sim_params["n_sites"], depth=sim_params["depth"],
            p_init=int(sim_params["n_sites"] * sim_params["p_init_factor"]),
            initial_coin_vector=initial_coin_ket0,
            static_substrate_pattern=fib_substrate,
            coin_for_0=cp_info["c0"], coin_for_1=cp_info["c1"],
            substrate_name="Fibonacci"
        )
        plot_results(results, save_path=os.path.join(output_base_dir, f"{sim_label}_N{sim_params['n_sites']}_D{sim_params['depth']}.png"),
                     plot_title_prefix=f"1W QW ({cp_info['name']})")

    # --- 2. Longer Evolution (Example with one coin pair) ---
    # This is implicitly tested by using larger DEPTH_DEFAULT or sim_params["depth"]

    # --- 3. Different Aperiodic Substrates (Thue-Morse) ---
    print("\n===== 3. Thue-Morse Substrate =====")
    tm_substrate = generate_thue_morse_sequence_01(sim_params["n_sites"])

    # Using H vs XH coins for Thue-Morse as an example
    sim_label_tm = "ThueMorse_H_vs_XH"
    print(f"\n--- Running: {sim_label_tm} ---")
    results_tm = run_1d_qw_1walker_aperiodic_substrate(
        n_sites_1d=sim_params["n_sites"], depth=sim_params["depth"],
        p_init=int(sim_params["n_sites"] * sim_params["p_init_factor"]),
        initial_coin_vector=initial_coin_ket0,
        static_substrate_pattern=tm_substrate,
        coin_for_0=H_1Q, coin_for_1=X_1Q @ H_1Q,
        substrate_name="ThueMorse"
    )
    plot_results(results_tm, save_path=os.path.join(output_base_dir, f"{sim_label_tm}_N{sim_params['n_sites']}_D{sim_params['depth']}.png"),
                 plot_title_prefix="1W QW (H vs XH)")

    # --- 4. Quantitative Fractal Analysis (Box-Counting) ---
    print("\n===== 4. Fractal Dimension Analysis (Box-Counting) =====")
    # Example: Analyze the last simulation (Thue-Morse) at a few time steps
    if results_tm: # Check if simulation ran
        prob_hist_tm = results_tm["prob_hist"]
        times_to_analyze = [sim_params["depth"] // 2, sim_params["depth"]] # Mid and final step
        for t_fixed in times_to_analyze:
            if t_fixed < prob_hist_tm.shape[0]:
                p_at_t = prob_hist_tm[t_fixed, :]
                print(f"\nAnalyzing {results_tm['params']['substrate_type']} at t={t_fixed}...")
                box_s, counts = box_counting_1d(p_at_t, threshold=1e-4) # Adjust threshold as needed
                if len(box_s) > 0:
                    plot_fractal_dimension_analysis(box_s, counts, results_tm['params']['substrate_type'], t_fixed, output_base_dir)
                else:
                    print(f"  No data for box counting at t={t_fixed}")
            else:
                print(f"  Time step t={t_fixed} is out of bounds for analysis.")


    # --- 5. Initial Coin State ---
    print("\n===== 5. Varying Initial Coin State (Fibonacci Substrate, H vs XH coins) =====")
    initial_coin_ket_plus = H_1Q @ np.array([1,0], dtype=np.complex128) # |+>_c state = (1/sqrt(2))(|0> + |1>)

    sim_label_init_coin = "Fibonacci_H_vs_XH_InitPlus"
    print(f"\n--- Running: {sim_label_init_coin} ---")
    results_init_plus = run_1d_qw_1walker_aperiodic_substrate(
        n_sites_1d=sim_params["n_sites"], depth=sim_params["depth"],
        p_init=int(sim_params["n_sites"] * sim_params["p_init_factor"]),
        initial_coin_vector=initial_coin_ket_plus, # Using the |+> state
        static_substrate_pattern=fib_substrate, # Reuse Fibonacci from earlier
        coin_for_0=H_1Q, coin_for_1=X_1Q @ H_1Q,
        substrate_name="Fibonacci"
    )
    plot_results(results_init_plus, save_path=os.path.join(output_base_dir, f"{sim_label_init_coin}_N{sim_params['n_sites']}_D{sim_params['depth']}.png"),
                 plot_title_prefix="1W QW (H vs XH, Init |+>)")

    print("\n<<<<< ALL APERIODIC QW EXPLORATIONS COMPLETE >>>>>")
    print(f"Plots saved in ./{output_base_dir}/")


===== 1. Varying Coins on Fibonacci Substrate =====

--- Running: Fibonacci_H_vs_XH ---
Initialized Single Walker at pos=75 with coin_vec=[1.000+0.000j, 0.000+0.000j]

Starting 1-Walker 1D QW on static Fibonacci Substrate (150 sites, 200 steps)...
  Fibonacci-QW Step 20/200 completed. S_coin=0.796
  Fibonacci-QW Step 40/200 completed. S_coin=0.956
  Fibonacci-QW Step 60/200 completed. S_coin=0.998
  Fibonacci-QW Step 80/200 completed. S_coin=0.457
  Fibonacci-QW Step 100/200 completed. S_coin=0.972
  Fibonacci-QW Step 120/200 completed. S_coin=0.921
  Fibonacci-QW Step 140/200 completed. S_coin=0.923
  Fibonacci-QW Step 160/200 completed. S_coin=0.956
  Fibonacci-QW Step 180/200 completed. S_coin=0.997
  Fibonacci-QW Step 200/200 completed. S_coin=0.929
1-Walker Fibonacci QW Evolution complete. Time: 13.64 seconds.
Plot saved to qw_aperiodic_explorations/Fibonacci_H_vs_XH_N150_D200.png

--- Running: Fibonacci_H_vs_I ---
Initialized Single Walker at pos=75 with coin_vec=[1.000+0.000j, 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import time
import os

# --- Parameters (can be overridden in main execution) ---
N_SITES_1D_DEFAULT = 100
DEPTH_DEFAULT = 150
# Initial positions and coins will be set per walker

# --- Coin Matrix Definitions (Unchanged) ---
H_1Q = (1/np.sqrt(2))*np.array([[1,1],[1,-1]],dtype=np.complex128)
X_1Q = np.array([[0,1],[1,0]],dtype=np.complex128)
I_1Q = np.eye(2, dtype=np.complex128)
Z_1Q = np.array([[1,0],[0,-1]], dtype=np.complex128)
Y_1Q = np.array([[0,-1j],[1j,0]], dtype=np.complex128)
def Rz_gate(phi):
    return np.array([[np.exp(-1j*phi/2), 0],
                     [0, np.exp(1j*phi/2)]], dtype=np.complex128)

# --- Aperiodic Sequence Generation (Unchanged) ---
def generate_fibonacci_sequence_01(length):
    s_n_minus_2_str = "0"; s_n_minus_1_str = "01"
    if length <= 0: return np.array([], dtype=int)
    if length == 1: return np.array([int(s_n_minus_2_str[0])], dtype=int)
    current_sequence_str = s_n_minus_1_str
    while len(current_sequence_str) < length:
        s_n_str = s_n_minus_1_str + s_n_minus_2_str
        current_sequence_str = s_n_str
        s_n_minus_2_str = s_n_minus_1_str; s_n_minus_1_str = s_n_str
    seq_int = np.array([int(c) for c in current_sequence_str[:length]], dtype=int)
    if len(seq_int) < length: seq_int = np.pad(seq_int, (0, length - len(seq_int)), 'wrap')
    return seq_int

def generate_thue_morse_sequence_01(length):
    if length <= 0: return np.array([], dtype=int)
    tm_seq = [0]
    while len(tm_seq) < length: tm_seq.extend([1 - x for x in tm_seq])
    return np.array(tm_seq[:length], dtype=int)

# --- Helper Functions for 1D QW State Indexing (Unchanged) ---
def get_1d_index(coin_val, site_pos, n_sites_1d):
    if not (0 <= site_pos < n_sites_1d and 0 <= coin_val <= 1):
        raise IndexError(f"Invalid coin ({coin_val}) or pos ({site_pos}) for 1D index (N_sites={n_sites_1d})")
    return coin_val + 2 * site_pos
def get_1d_coin_pos_from_index(k, n_sites_1d):
    state_dim_1d = 2 * n_sites_1d
    if not (0 <= k < state_dim_1d):
        raise IndexError(f"Invalid k ({k}) for 1D coin/pos (StateDim={state_dim_1d})")
    coin_val = k % 2; site_pos = k // 2
    return coin_val, site_pos

# --- Initial State Preparation (Flexible - Unchanged) ---
def prepare_initial_state_1walker_flexible(n_sites_1d, p_init, initial_coin_vector, walker_label=""):
    state_dim_1d = 2 * n_sites_1d
    initial_state_vector = np.zeros(state_dim_1d, dtype=np.complex128)
    if len(initial_coin_vector) != 2: raise ValueError("initial_coin_vector must have 2 elements.")
    if not np.isclose(np.linalg.norm(initial_coin_vector), 1.0):
        print(f"Warning: Initial coin vector for Walker {walker_label} is not normalized. Normalizing...")
        initial_coin_vector = initial_coin_vector / np.linalg.norm(initial_coin_vector)
    try:
        idx_c0 = get_1d_index(0, p_init, n_sites_1d)
        idx_c1 = get_1d_index(1, p_init, n_sites_1d)
        initial_state_vector[idx_c0] = initial_coin_vector[0]
        initial_state_vector[idx_c1] = initial_coin_vector[1]
        print(f"Initialized Walker {walker_label} at pos={p_init} with coin_vec=[{initial_coin_vector[0]:.3f}, {initial_coin_vector[1]:.3f}]")
    except IndexError as e: print(f"Error during Walker {walker_label} initial state prep: {e}")
    return initial_state_vector

# --- Build 1D QW Operators (Unchanged) ---
def build_single_qw_coin_matrix_for_site(substrate_site_val, coin_for_0, coin_for_1):
    return coin_for_0 if substrate_site_val == 0 else coin_for_1
def build_1walker_coin_operator(n_sites_1d, static_substrate_pattern, coin_for_0, coin_for_1):
    d_1w = 2 * n_sites_1d; U_C = np.zeros((d_1w, d_1w), dtype=np.complex128)
    for site_idx in range(n_sites_1d):
        coin_op_for_this_site = build_single_qw_coin_matrix_for_site(static_substrate_pattern[site_idx], coin_for_0, coin_for_1)
        U_C[2*site_idx : 2*site_idx+2, 2*site_idx : 2*site_idx+2] = coin_op_for_this_site
    return U_C
def build_single_qw_shift_matrix(n_sites_1d):
    d_1w = 2 * n_sites_1d; S_1w = np.zeros((d_1w, d_1w), dtype=np.complex128)
    for k_in_1w in range(d_1w):
        coin_val, site_pos = get_1d_coin_pos_from_index(k_in_1w, n_sites_1d)
        if coin_val == 0: new_pos = (site_pos - 1 + n_sites_1d) % n_sites_1d
        else: new_pos = (site_pos + 1) % n_sites_1d
        k_out_1w = get_1d_index(coin_val, new_pos, n_sites_1d)
        S_1w[k_out_1w, k_in_1w] = 1.0
    return S_1w

# --- Observables for Single Walker (Unchanged) ---
def calculate_p_pos_1walker(qw_state, n_sites_1d):
    d_1w = 2 * n_sites_1d; probs_1d = np.zeros(n_sites_1d)
    if len(qw_state) != d_1w: return probs_1d
    for k in range(d_1w):
        _, site_pos = get_1d_coin_pos_from_index(k, n_sites_1d)
        probs_1d[site_pos] += np.abs(qw_state[k])**2
    return probs_1d
def calculate_coin_entanglement_1walker(qw_state, n_sites_1d):
    d_1w = 2 * n_sites_1d; coin_dim = 2
    if len(qw_state) != d_1w: return np.nan
    norm = np.linalg.norm(qw_state)
    if norm < 1e-9: return 0.0
    current_psi = qw_state / norm if np.abs(norm - 1.0) > 1e-6 else qw_state
    rho_full = np.outer(current_psi, np.conjugate(current_psi))
    rho_coin_reduced = np.zeros((coin_dim, coin_dim), dtype=np.complex128)
    for k_row in range(d_1w):
        c_r, p_r = get_1d_coin_pos_from_index(k_row, n_sites_1d)
        for k_col in range(d_1w):
            c_c, p_c = get_1d_coin_pos_from_index(k_col, n_sites_1d)
            if p_r == p_c: rho_coin_reduced[c_r, c_c] += rho_full[k_row, k_col]
    tr_rho_coin = np.trace(rho_coin_reduced)
    if abs(tr_rho_coin) < 1e-9: return 0.0
    if np.abs(tr_rho_coin - 1.0) > 1e-6 : rho_coin_reduced /= tr_rho_coin
    eigs = np.linalg.eigvalsh(rho_coin_reduced); ent = 0.0; epsilon = 1e-12
    for e_val in eigs: ent -= (e_val * np.log2(e_val) if e_val > epsilon else 0.0)
    return max(0.0, np.real(ent))

# --- Simulation Loop (Modified to accept walker_label) ---
def run_1d_qw_1walker_aperiodic_substrate(
    n_sites_1d, depth,
    p_init, initial_coin_vector,
    static_substrate_pattern,
    coin_for_0, coin_for_1,
    substrate_name="Aperiodic", walker_label=""
):
    current_qw_state = prepare_initial_state_1walker_flexible(n_sites_1d, p_init, initial_coin_vector, walker_label)
    prob_hist = np.full((depth + 1, n_sites_1d), np.nan)
    ent_hist = np.full(depth + 1, np.nan)
    substrate_pattern_history = [static_substrate_pattern.copy()] # Store once for plotting
    prob_hist[0, :] = calculate_p_pos_1walker(current_qw_state, n_sites_1d)
    ent_hist[0] = calculate_coin_entanglement_1walker(current_qw_state, n_sites_1d)

    print(f"\nStarting Walker {walker_label} QW on static {substrate_name} Substrate ({n_sites_1d} sites, {depth} steps)...")
    start_time = time.time()
    S_Shift_constant = build_single_qw_shift_matrix(n_sites_1d)
    U_Coin_constant = build_1walker_coin_operator(n_sites_1d, static_substrate_pattern, coin_for_0, coin_for_1)
    U_Step_constant = S_Shift_constant @ U_Coin_constant

    for step in range(depth):
        current_qw_state = U_Step_constant @ current_qw_state
        norm_qw = np.linalg.norm(current_qw_state)
        if abs(norm_qw) > 1e-9: current_qw_state /= norm_qw
        else: print(f"Warning: Walker {walker_label} QW Norm zero at step {step+1}."); break
        prob_hist[step + 1, :] = calculate_p_pos_1walker(current_qw_state, n_sites_1d)
        ent_hist[step+1] = calculate_coin_entanglement_1walker(current_qw_state, n_sites_1d)
        # No need to append substrate_pattern_history in loop as it's static

        if (step + 1) % (max(1,depth // 10)) == 0 or step == depth -1:
            print(f"  Walker {walker_label} {substrate_name}-QW Step {step + 1}/{depth} completed. S_coin={ent_hist[step+1]:.3f}")
    end_time = time.time()
    print(f"Walker {walker_label} {substrate_name} QW Evolution complete. Time: {end_time - start_time:.2f} seconds.")
    return {
        "prob_hist": prob_hist, "ent_hist": ent_hist,
        "ca_history": np.array(substrate_pattern_history), # Static substrate
        "params": { "n_sites":n_sites_1d,"depth":depth, "substrate_type": substrate_name,
                    "p_init":p_init, "initial_coin_vector_str": str(initial_coin_vector),
                    "coin_0_info": "UserDefined", "coin_1_info": "UserDefined", "walker_label": walker_label }
    }

# --- Plotting for Two Independent Walkers ---
def plot_two_independent_walkers_results(results_A, results_B, save_path=None, plot_title_prefix="Two Indep. QWs"):
    params_A = results_A["params"]; p_hist_A = results_A["prob_hist"]; e_hist_A = results_A["ent_hist"]
    params_B = results_B["params"]; p_hist_B = results_B["prob_hist"]; e_hist_B = results_B["ent_hist"]

    # Assume substrate is the same for both, use A's
    static_substrate = results_A["ca_history"][0] # It's static
    n_sites = params_A["n_sites"]; depth = params_A["depth"]
    substrate_info = params_A.get("substrate_type", "Unknown Substrate")

    fig, axs = plt.subplots(5, 1, figsize=(12, 22), gridspec_kw={'height_ratios': [1.0, 3, 3, 2, 2]})
    title_str = f"{plot_title_prefix} on {substrate_info} (N={n_sites}, D={depth})"
    fig.suptitle(title_str, fontsize=14)

    # 1. Static Substrate
    axs[0].imshow(static_substrate.reshape(1, -1), cmap='binary', aspect='auto', interpolation='nearest', extent=[0, n_sites-1, 0, 1])
    axs[0].set_title(f"Static {substrate_info} Substrate Pattern (0/1)"); axs[0].set_xlabel("Site Index"); axs[0].set_yticks([])

    time_extent = [0, depth, 0, n_sites-1]
    norm_v = None # Linear scale for probability

    # 2. P(x,t) for Walker A
    prob_st_A=p_hist_A.T
    im_A=axs[1].imshow(prob_st_A, aspect='auto', origin='lower', cmap='viridis', norm=norm_v, extent=time_extent)
    plt.colorbar(im_A, ax=axs[1], label="P_A(x,t)"); axs[1].set_title("Walker A P(x,t)"); axs[1].set_xlabel("Time Step"); axs[1].set_ylabel("Site Index (x)")

    # 3. P(x,t) for Walker B
    prob_st_B=p_hist_B.T
    im_B=axs[2].imshow(prob_st_B, aspect='auto', origin='lower', cmap='magma', norm=norm_v, extent=time_extent) # Different colormap
    plt.colorbar(im_B, ax=axs[2], label="P_B(x,t)"); axs[2].set_title("Walker B P(x,t)"); axs[2].set_xlabel("Time Step"); axs[2].set_ylabel("Site Index (x)")

    ts=np.arange(depth+1)
    # 4. Coin Entanglement
    axs[3].plot(ts, e_hist_A, marker='.', ls='-', color='blue', label='Walker A Coin Ent.')
    axs[3].plot(ts, e_hist_B, marker='.', ls='--', color='red', label='Walker B Coin Ent.') # Dashed line for B
    axs[3].set_title("Coin Entanglement S(t)"); axs[3].set_xlabel("Time Step"); axs[3].set_ylabel("Entropy S (bits)"); axs[3].grid(True); axs[3].legend(); axs[3].set_ylim(bottom=-0.05, top=1.05)

    # 5. Average Position
    avg_pos_A = np.array([np.sum(p_hist_A[t,:] * np.arange(n_sites)) / np.sum(p_hist_A[t,:]) if np.sum(p_hist_A[t,:]) > 1e-9 else np.nan for t in range(depth+1)])
    avg_pos_B = np.array([np.sum(p_hist_B[t,:] * np.arange(n_sites)) / np.sum(p_hist_B[t,:]) if np.sum(p_hist_B[t,:]) > 1e-9 else np.nan for t in range(depth+1)])
    axs[4].plot(ts, avg_pos_A, marker='.', ls='-', color='green', label='Walker A <x>')
    axs[4].plot(ts, avg_pos_B, marker='.', ls='--', color='purple', label='Walker B <x>') # Dashed line for B
    axs[4].set_title("Average Position <x>(t)"); axs[4].set_xlabel("Time Step"); axs[4].set_ylabel("<x>"); axs[4].grid(True); axs[4].legend()

    plt.tight_layout(rect=[0,0,1,0.96])
    if save_path: plt.savefig(save_path); print(f"Plot saved to {save_path}"); plt.close(fig)
    else: plt.show()

# --- Box-Counting and Plotting (Unchanged) ---
def box_counting_1d(data_array, threshold=1e-5):
    occupied_sites = np.where(data_array > threshold)[0]
    if len(occupied_sites) == 0: print("No occupied sites for box-counting."); return np.array([]), np.array([])
    min_coord = np.min(occupied_sites); max_coord = np.max(occupied_sites)
    data_range = max_coord - min_coord + 1
    if data_range <= 0: data_range = 1
    max_power = int(np.log2(data_range));  max_power = max(1, max_power)
    box_sizes = np.array([2**i for i in range(max_power + 1) if 2**i <= data_range/2 and 2**i > 0])
    if len(box_sizes) == 0:
        box_sizes = np.array([1,2,4]) if data_range >=4 else np.array([1])
        box_sizes = box_sizes[box_sizes <= data_range/2]
        if len(box_sizes) == 0 and data_range > 0 : box_sizes = np.array([1])
    counts = []
    for s_val in box_sizes:
        s = int(s_val); num_boxes_occupied = 0; current_pos = min_coord
        while current_pos <= max_coord:
            box_end = current_pos + s -1
            if np.any((occupied_sites >= current_pos) & (occupied_sites <= box_end)): num_boxes_occupied += 1
            current_pos += s
        counts.append(num_boxes_occupied)
    return np.array(box_sizes), np.array(counts)

def plot_fractal_dimension_analysis(box_sizes, counts, substrate_name, t_fixed, walker_label, save_dir):
    if len(box_sizes) < 2 or len(counts) < 2:
        print(f"Not enough data for fractal dim plot ({substrate_name}, Walker {walker_label}, t={t_fixed})."); return
    log_inv_box_sizes = np.log(1.0 / box_sizes); log_counts = np.log(counts)
    plt.figure(figsize=(8, 6))
    plt.plot(log_inv_box_sizes, log_counts, 'o-', label=f'{substrate_name} W:{walker_label} t={t_fixed}')
    valid_indices = np.where(counts > 1)[0]
    if len(valid_indices) > 1 :
        slope, intercept = np.polyfit(log_inv_box_sizes[valid_indices], log_counts[valid_indices], 1)
        plt.plot(log_inv_box_sizes, slope * log_inv_box_sizes + intercept, '--', label=f'Fit (Df ≈ {slope:.3f})')
        print(f"Est. Fractal Dim Df for {substrate_name} W:{walker_label} t={t_fixed}: {slope:.3f}")
    else: print(f"Could not fit for {substrate_name} W:{walker_label} t={t_fixed} (N(s)<=1).")
    plt.xlabel("log(1 / Box Size)"); plt.ylabel("log(Number of Occupied Boxes)")
    plt.title(f"Fractal Dimension Analysis (Box-Counting)"); plt.legend(); plt.grid(True)
    if save_dir:
        plt.savefig(os.path.join(save_dir, f"fractal_dim_{substrate_name}_W{walker_label}_t{t_fixed}.png"))
        print(f"Fractal dim plot saved for {substrate_name} W:{walker_label} t={t_fixed}."); plt.close()
    else: plt.show()


# --- Main Execution Script ---
if __name__ == "__main__":
    output_base_dir = "qw_aperiodic_2walker_explorations"
    if not os.path.exists(output_base_dir): os.makedirs(output_base_dir)

    sim_params = {
        "n_sites": 120,  # Adjusted
        "depth": 180,    # Adjusted
    }

    # Define initial conditions for two walkers
    p_init_A = sim_params["n_sites"] // 3
    coin_vec_A = np.array([1,0], dtype=np.complex128) # |0>_c for Walker A

    p_init_B = 2 * sim_params["n_sites"] // 3
    coin_vec_B = H_1Q @ np.array([1,0], dtype=np.complex128) # |+>_c for Walker B (different)

    # --- Experiment: Two walkers on Fibonacci with H vs XH coins ---
    print("\n===== Two Walkers on Fibonacci Substrate (H vs XH) =====")
    substrate_pattern = generate_fibonacci_sequence_01(sim_params["n_sites"])
    coin0_choice = H_1Q
    coin1_choice = X_1Q @ H_1Q
    substrate_name_current = "Fibonacci_HvsXH"

    print("\n--- Running Walker A ---")
    results_A = run_1d_qw_1walker_aperiodic_substrate(
        n_sites_1d=sim_params["n_sites"], depth=sim_params["depth"],
        p_init=p_init_A, initial_coin_vector=coin_vec_A,
        static_substrate_pattern=substrate_pattern,
        coin_for_0=coin0_choice, coin_for_1=coin1_choice,
        substrate_name="Fibonacci", walker_label="A"
    )
    print("\n--- Running Walker B ---")
    results_B = run_1d_qw_1walker_aperiodic_substrate(
        n_sites_1d=sim_params["n_sites"], depth=sim_params["depth"],
        p_init=p_init_B, initial_coin_vector=coin_vec_B,
        static_substrate_pattern=substrate_pattern, # Same substrate
        coin_for_0=coin0_choice, coin_for_1=coin1_choice, # Same coin rules
        substrate_name="Fibonacci", walker_label="B"
    )

    plot_two_independent_walkers_results(results_A, results_B,
        save_path=os.path.join(output_base_dir, f"TwoWalkers_{substrate_name_current}_N{sim_params['n_sites']}_D{sim_params['depth']}.png"),
        plot_title_prefix="Two Indep. QWs")

    # --- Perform fractal analysis for each walker from the above simulation ---
    print("\n===== Fractal Dimension Analysis for Two Walkers =====")
    times_to_analyze = [sim_params["depth"] // 2, sim_params["depth"]]
    for t_fixed in times_to_analyze:
        if results_A and t_fixed < results_A["prob_hist"].shape[0]:
            print(f"\nAnalyzing Walker A on {substrate_name_current} at t={t_fixed}...")
            box_s_A, counts_A = box_counting_1d(results_A["prob_hist"][t_fixed, :], threshold=1e-4)
            if len(box_s_A) > 0:
                plot_fractal_dimension_analysis(box_s_A, counts_A, substrate_name_current, t_fixed, "A", output_base_dir)

        if results_B and t_fixed < results_B["prob_hist"].shape[0]:
            print(f"\nAnalyzing Walker B on {substrate_name_current} at t={t_fixed}...")
            box_s_B, counts_B = box_counting_1d(results_B["prob_hist"][t_fixed, :], threshold=1e-4)
            if len(box_s_B) > 0:
                plot_fractal_dimension_analysis(box_s_B, counts_B, substrate_name_current, t_fixed, "B", output_base_dir)

    # --- You can now re-introduce the loops for varying coins, substrates, etc. ---
    # For each experimental setup, you would:
    # 1. Define substrate_pattern, coin0_choice, coin1_choice.
    # 2. Run Walker A with these settings.
    # 3. Run Walker B with these settings.
    # 4. Call plot_two_independent_walkers_results(results_A, results_B, ...).
    # Example for varying coins (only one pair shown for brevity):
    # coin_pairs_to_test = [{"name": "H_vs_I", "c0": H_1Q, "c1": I_1Q}]
    # for cp_info in coin_pairs_to_test:
    #    substrate_name_current = f"Fibonacci_{cp_info['name']}"
    #    # ... run A, run B, plot_two_independent_walkers_results ...


    print("\n<<<<< ALL 2-WALKER (INDEPENDENT) APERIODIC QW EXPLORATIONS SETUP >>>>>")
    print(f"Plots and analysis saved in ./{output_base_dir}/")


===== Two Walkers on Fibonacci Substrate (H vs XH) =====

--- Running Walker A ---
Initialized Walker A at pos=40 with coin_vec=[1.000+0.000j, 0.000+0.000j]

Starting Walker A QW on static Fibonacci Substrate (120 sites, 180 steps)...
  Walker A Fibonacci-QW Step 18/180 completed. S_coin=0.910
  Walker A Fibonacci-QW Step 36/180 completed. S_coin=0.833
  Walker A Fibonacci-QW Step 54/180 completed. S_coin=0.818
  Walker A Fibonacci-QW Step 72/180 completed. S_coin=0.695
  Walker A Fibonacci-QW Step 90/180 completed. S_coin=0.861
  Walker A Fibonacci-QW Step 108/180 completed. S_coin=0.923
  Walker A Fibonacci-QW Step 126/180 completed. S_coin=0.982
  Walker A Fibonacci-QW Step 144/180 completed. S_coin=0.993
  Walker A Fibonacci-QW Step 162/180 completed. S_coin=0.967
  Walker A Fibonacci-QW Step 180/180 completed. S_coin=0.991
Walker A Fibonacci QW Evolution complete. Time: 6.35 seconds.

--- Running Walker B ---
Initialized Walker B at pos=80 with coin_vec=[0.707+0.000j, 0.707+0.000

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import time
import os

# --- Parameters ---
N_SITES_1D = 30 # Keep manageable for 2 walkers
DEPTH = 50
INITIAL_POS_A = N_SITES_1D // 2 - N_SITES_1D // 6
INITIAL_POS_B = N_SITES_1D // 2 + N_SITES_1D // 6
INITIAL_COIN_A_VEC = np.array([1,0], dtype=np.complex128) # |0>
INITIAL_COIN_B_VEC = np.array([1,0], dtype=np.complex128) # |0>
# (Substrate type will be Fibonacci)

# --- Coins for the substrate ---
H_1Q = (1/np.sqrt(2))*np.array([[1,1],[1,-1]],dtype=np.complex128)
X_1Q = np.array([[0,1],[1,0]],dtype=np.complex128)
I_1Q = np.eye(2, dtype=np.complex128)

COIN_SUBSTRATE_0 = H_1Q
COIN_SUBSTRATE_1 = X_1Q @ H_1Q

# --- Interaction Gate (Optional) ---
# This gate is applied to the coin Hilbert space of the two walkers if they are at the same site.
# Example: CNOT gate (control A, target B)
CNOT_2Q = np.array([[1,0,0,0], [0,1,0,0], [0,0,0,1], [0,0,1,0]], dtype=np.complex128)
# Example: SWAP gate
SWAP_2Q = np.array([[1,0,0,0], [0,0,1,0], [0,1,0,0], [0,0,0,1]], dtype=np.complex128)
# If no interaction, use Identity on the 2-qubit coin space
NO_INTERACTION_2Q = np.eye(4, dtype=np.complex128)

# CHOOSE INTERACTION:
INTERACTION_GATE_2Q = NO_INTERACTION_2Q # Or CNOT_2Q, SWAP_2Q
INTERACTION_ENABLED = False if np.array_equal(INTERACTION_GATE_2Q, NO_INTERACTION_2Q) else True


# --- Aperiodic Sequence Generation (Fibonacci) ---
def generate_fibonacci_sequence_01(length):
    s_n_minus_2_str = "0"; s_n_minus_1_str = "01"
    if length <= 0: return np.array([], dtype=int)
    if length == 1: return np.array([int(s_n_minus_2_str[0])], dtype=int)
    current_sequence_str = s_n_minus_1_str
    while len(current_sequence_str) < length:
        s_n_str = s_n_minus_1_str + s_n_minus_2_str
        current_sequence_str = s_n_str
        s_n_minus_2_str = s_n_minus_1_str; s_n_minus_1_str = s_n_str
    seq_int = np.array([int(c) for c in current_sequence_str[:length]], dtype=int)
    if len(seq_int) < length: seq_int = np.pad(seq_int, (0, length - len(seq_int)), 'wrap')
    return seq_int

# --- Helper Functions (1D QW - Single Walker Indexing - Unchanged) ---
def get_1d_index(coin_val, site_pos, n_sites_1d):
    # ... (same as before)
    if not (0 <= site_pos < n_sites_1d and 0 <= coin_val <= 1):
        raise IndexError(f"Invalid coin ({coin_val}) or pos ({site_pos}) for 1D index (N_sites={n_sites_1d})")
    return coin_val + 2 * site_pos

def get_1d_coin_pos_from_index(k, n_sites_1d):
    # ... (same as before)
    state_dim_1d = 2 * n_sites_1d
    if not (0 <= k < state_dim_1d):
        raise IndexError(f"Invalid k ({k}) for 1D coin/pos (StateDim={state_dim_1d})")
    coin_val = k % 2; site_pos = k // 2
    return coin_val, site_pos

# --- Helper Functions (2 Walkers on 1D Grid - Unchanged from your original 2-walker code) ---
def get_2walker_index(cA, pA, cB, pB, n_sites_1d):
    d_1w = 2 * n_sites_1d
    idx_A = get_1d_index(cA, pA, n_sites_1d)
    idx_B = get_1d_index(cB, pB, n_sites_1d)
    if not (0 <= idx_A < d_1w and 0 <= idx_B < d_1w):
        raise IndexError(f"Internal 1D index out of bounds during 2-walker indexing.")
    return idx_A + d_1w * idx_B # A is faster index

def get_2walker_coords_from_index(k, n_sites_1d):
    d_1w = 2 * n_sites_1d
    d_tot = d_1w * d_1w
    if not (0 <= k < d_tot):
        raise IndexError(f"Invalid k ({k}) for 2-walker (Dim={d_tot})")
    idx_A = k % d_1w
    idx_B = k // d_1w
    cA, pA = get_1d_coin_pos_from_index(idx_A, n_sites_1d)
    cB, pB = get_1d_coin_pos_from_index(idx_B, n_sites_1d)
    return cA, pA, cB, pB

# --- Initial State Preparation (2 Walkers, flexible coin) ---
def prepare_initial_state_2walkers_flexible(n_sites_1d, pA_init, cA_vec, pB_init, cB_vec):
    d_1w = 2 * n_sites_1d
    state_dim_2w = d_1w * d_1w
    initial_state_vector = np.zeros(state_dim_2w, dtype=np.complex128)

    # Create full initial state by Kronecker product of individual initial states
    psi_A_init = np.zeros(d_1w, dtype=np.complex128)
    psi_A_init[get_1d_index(0, pA_init, n_sites_1d)] = cA_vec[0]
    psi_A_init[get_1d_index(1, pA_init, n_sites_1d)] = cA_vec[1]

    psi_B_init = np.zeros(d_1w, dtype=np.complex128)
    psi_B_init[get_1d_index(0, pB_init, n_sites_1d)] = cB_vec[0]
    psi_B_init[get_1d_index(1, pB_init, n_sites_1d)] = cB_vec[1]

    # Combine: |psi_B> kron |psi_A> (since A is faster index in get_2walker_index)
    full_initial_state_temp = np.kron(psi_B_init, psi_A_init)

    # The get_2walker_index maps |cA,pA,cB,pB> so we need to be careful or just use a simpler way
    # Simpler: find the single basis state |cA_0, pA_0, cB_0, pB_0> and set its amplitude.
    # This assumes starting in a product state of definite coin states.
    # For general cA_vec, cB_vec, we need to sum over the coin basis:
    for cA_val in range(2):
        for cB_val in range(2):
            amp = cA_vec[cA_val] * cB_vec[cB_val]
            if np.abs(amp) > 1e-9:
                try:
                    idx = get_2walker_index(cA_val, pA_init, cB_val, pB_init, n_sites_1d)
                    initial_state_vector[idx] = amp
                except IndexError: pass # Should not happen if pA_init, pB_init are valid

    if not np.isclose(np.linalg.norm(initial_state_vector), 1.0) and np.linalg.norm(initial_state_vector) > 1e-9:
        print("Normalizing initial 2-walker state vector.")
        initial_state_vector /= np.linalg.norm(initial_state_vector)

    print(f"Initialized 2-Walker state. A at {pA_init} (coin ~{cA_vec}), B at {pB_init} (coin ~{cB_vec})")
    return initial_state_vector


# --- Build 2-Walker Operators on Static Substrate ---
def build_2walker_coin_operators_on_substrate(n_sites_1d, static_substrate, c_sub_0, c_sub_1):
    d_1w = 2 * n_sites_1d
    # Coin operator for Walker A (acts as C_A x I_B)
    U_C_A_single_walker_space = np.zeros((d_1w, d_1w), dtype=np.complex128)
    for site_idx in range(n_sites_1d):
        coin_op = c_sub_0 if static_substrate[site_idx] == 0 else c_sub_1
        U_C_A_single_walker_space[2*site_idx : 2*site_idx+2, 2*site_idx : 2*site_idx+2] = coin_op
    U_Coin_A_global = np.kron(np.eye(d_1w), U_C_A_single_walker_space) # I_B kron U_C_A(p_A) (if A is faster index)
                                                                    # Order matters for kronecker
                                                                    # Our get_2walker_index has A as faster index.
                                                                    # So state is |pB, cB, pA, cA> effectively
                                                                    # U_Coin_A should be I_B kron U_Coin_A_local
                                                                    # U_Coin_B should be U_Coin_B_local kron I_A
    # Let's use the explicit loop method from your original 2-walker code to be sure.
    state_dim_2w = d_1w * d_1w
    U_C_A = np.zeros((state_dim_2w, state_dim_2w), dtype=np.complex128)
    U_C_B = np.zeros((state_dim_2w, state_dim_2w), dtype=np.complex128)

    for k_in_joint in range(state_dim_2w):
        cA_in, pA_in, cB_in, pB_in = get_2walker_coords_from_index(k_in_joint, n_sites_1d)

        # Coin A
        coin_matrix_A = c_sub_0 if static_substrate[pA_in] == 0 else c_sub_1
        for cA_out in range(2):
            amp_A = coin_matrix_A[cA_out, cA_in]
            if np.abs(amp_A) > 1e-9:
                k_out_A = get_2walker_index(cA_out, pA_in, cB_in, pB_in, n_sites_1d)
                U_C_A[k_out_A, k_in_joint] = amp_A

        # Coin B
        coin_matrix_B = c_sub_0 if static_substrate[pB_in] == 0 else c_sub_1
        for cB_out in range(2):
            amp_B = coin_matrix_B[cB_out, cB_in]
            if np.abs(amp_B) > 1e-9:
                k_out_B = get_2walker_index(cA_in, pA_in, cB_out, pB_in, n_sites_1d)
                U_C_B[k_out_B, k_in_joint] = amp_B
    return U_C_A, U_C_B

def build_2walker_shift_operators(n_sites_1d):
    d_1w = 2 * n_sites_1d
    S_1w = build_single_qw_shift_matrix(n_sites_1d) # Reusing single walker shift
    # S_A acts on A's space, Identity on B's space. A is faster index.
    S_A_global = np.kron(np.eye(d_1w), S_1w)
    # S_B acts on B's space, Identity on A's space.
    S_B_global = np.kron(S_1w, np.eye(d_1w))
    return S_A_global, S_B_global

def build_2walker_interaction_operator(n_sites_1d, interaction_gate_2q):
    d_1w = 2 * n_sites_1d
    state_dim_2w = d_1w * d_1w
    U_int = np.eye(state_dim_2w, dtype=np.complex128) # Start with identity

    if np.array_equal(interaction_gate_2q, np.eye(4)): # No interaction
        return U_int

    for p_common in range(n_sites_1d): # Iterate over all possible meeting sites
        # Construct basis states for coins at p_common: |cAcB>
        # cA=0,cB=0 -> index 0 in 2Q space
        # cA=1,cB=0 -> index 1
        # cA=0,cB=1 -> index 2
        # cA=1,cB=1 -> index 3
        # Our 2-walker index: get_2walker_index(cA, p_common, cB, p_common, n_sites_1d)

        # Matrix for this specific site p_common
        # This sub-matrix will be (4x4) acting on the coin Hilbert space when pA=pB=p_common
        # The full U_int needs to pick out these 4x4 blocks.

        # Find all k_in states where pA == pB == p_common
        indices_at_p_common = []
        coin_configs_at_p_common = [] # Store (cA, cB)
        for cA_val in range(2):
            for cB_val in range(2):
                k = get_2walker_index(cA_val, p_common, cB_val, p_common, n_sites_1d)
                indices_at_p_common.append(k)
                coin_configs_at_p_common.append((cA_val, cB_val))

        # Apply the 4x4 interaction_gate_2q to these specific indices
        for r in range(4): # Row in interaction_gate_2q (output coin config)
            for c in range(4): # Col in interaction_gate_2q (input coin config)
                amp_int = interaction_gate_2q[r, c]
                if np.abs(amp_int) > 1e-9:
                    # Map r, c (0..3) to (cA_out, cB_out) and (cA_in, cB_in)
                    # cA is faster index in 2Q space for CNOT, SWAP definition (usually C_target kron C_control)
                    # Let's say interaction_gate_2q acts on |cB_coin, cA_coin> basis for standard CNOT (A controls B)
                    # cA_in_2q  = c % 2
                    # cB_in_2q  = c // 2
                    # cA_out_2q = r % 2
                    # cB_out_2q = r // 2

                    # If interaction_gate_2q acts on |cA, cB> (A coin state | B coin state)
                    # cB_in_2q = c % 2
                    # cA_in_2q = c // 2
                    # cB_out_2q = r % 2
                    # cA_out_2q = r // 2
                    # This mapping must be consistent with how interaction_gate_2q is defined!
                    # Let's assume interaction_gate_2q has cA as the first qubit (control for CNOT), cB as second (target for CNOT)
                    # Basis: |00>_AB, |01>_AB, |10>_AB, |11>_AB (A is LSB for indexing 0..3)
                    # cA_in_val  = coin_configs_at_p_common[c][0]
                    # cB_in_val  = coin_configs_at_p_common[c][1]
                    # cA_out_val = coin_configs_at_p_common[r][0] # This is not right.
                    # cB_out_val = coin_configs_at_p_common[r][1]

                    # Correct input index in the full state space:
                    k_in_val = indices_at_p_common[c] # k_in for (cA_in_orig, cB_in_orig) at p_common

                    # Determine output coin states based on interaction_gate_2q's action on (cA_in_orig, cB_in_orig)
                    # This is the tricky part: mapping the 4x4 interaction_gate_2q's effect
                    # back to the full Hilbert space.
                    # It's easier if we think of extracting the 4x4 sub-block from psi, applying interaction_gate_2q,
                    # and putting it back. For the operator, we modify U_int.

                    # We need to map the (cA, cB) from coin_configs_at_p_common
                    # to the 0..3 index for the 4x4 interaction gate.
                    # Let coin_configs_at_p_common[c] be (cA_current, cB_current)
                    # Let coin_index_for_gate = cA_current * 2 + cB_current (if cB is faster for gate)
                    # Or coin_index_for_gate = cB_current * 2 + cA_current (if cA is faster for gate)
                    # Assume standard: A is q0 (LSB), B is q1 (MSB) for a 4x4 matrix indices 00, 01, 10, 11
                    idx_for_gate_in = coin_configs_at_p_common[c][0] + 2 * coin_configs_at_p_common[c][1] # (cA, cB) -> cA + 2*cB
                    idx_for_gate_out = r # This 'r' IS the output config index (0 to 3)

                    # Map idx_for_gate_out back to (cA_out, cB_out)
                    cA_out_val_from_gate = idx_for_gate_out % 2
                    cB_out_val_from_gate = idx_for_gate_out // 2

                    k_out_val = get_2walker_index(cA_out_val_from_gate, p_common, cB_out_val_from_gate, p_common, n_sites_1d)

                    # U_int[k_out, k_in] = amp
                    # Important: U_int starts as Identity. We are *replacing* entries.
                    # If c_in != c_out for the gate, the original Identity entry U_int[k_in, k_in] might need to be zeroed.
                    # This is simpler if we build the block and insert it.

                    # For each k_in = indices_at_p_common[c_idx_in_gate]
                    # For each k_out = indices_at_p_common[c_idx_out_gate]
                    # U_int[k_out, k_in] = interaction_gate_2q[c_idx_out_gate, c_idx_in_gate]
                    # This has to be done carefully.
                    # Let's iterate through all pairs of coin configurations (cA_in,cB_in) and (cA_out,cB_out) at p_common

                    # Correct way:
                    # For a fixed p_common, consider the 4x4 subspace of U_int
                    # U_int[np.ix_(indices_at_p_common, indices_at_p_common)] should become interaction_gate_2q
                    # BUT the order of indices_at_p_common might not match the basis order of interaction_gate_2q
                    # For interaction_gate_2q, let's assume basis |00>_AB, |01>_AB, |10>_AB, |11>_AB
                    # where A is LSB. Index = cA + 2*cB.

                    # Map from 0..3 (gate_idx_in) to k_in
                    cA_in_gate = c % 2
                    cB_in_gate = c // 2
                    k_in_global = get_2walker_index(cA_in_gate, p_common, cB_in_gate, p_common, n_sites_1d)

                    # Map from 0..3 (gate_idx_out) to k_out
                    cA_out_gate = r % 2
                    cB_out_gate = r // 2
                    k_out_global = get_2walker_index(cA_out_gate, p_common, cB_out_gate, p_common, n_sites_1d)

                    # Set U_int[k_out_global, k_in_global] to interaction_gate_2q[r, c]
                    # Must zero out the original identity if k_out_global != k_in_global for this component
                    if k_out_global == k_in_global: # Diagonal term of interaction gate
                        U_int[k_out_global, k_in_global] = interaction_gate_2q[r,c]
                    else: # Off-diagonal term
                        U_int[k_out_global, k_in_global] = interaction_gate_2q[r,c]
                        if U_int[k_in_global, k_in_global] == 1.0 : # If it was an identity element, and this k_in maps elsewhere
                             # This needs to be handled by ensuring that if interaction_gate_2q[c,c] is not 1,
                             # the diagonal U_int[k_in_global, k_in_global] is set to that.
                             # If interaction_gate_2q[c,c] is 0, then U_int[k_in_global, k_in_global] becomes 0.
                             pass # Will be correctly set by the loop if r==c

    # A more robust way for interaction part:
    # Iterate k_in. If pA_in == pB_in:
    #   Extract the 2-qbit coin state |cA_in, cB_in>. Map to index j for interaction_gate_2q.
    #   Apply interaction_gate_2q: Sum_i interaction_gate_2q[i,j] |coin_state_i>
    #   For each |coin_state_i> = |cA_out, cB_out>, get k_out. U_int[k_out, k_in] = interaction_gate_2q[i,j]
    # This is what your original 2-walker coin operator build did. Let's adapt that.

    U_interaction_final = np.zeros_like(U_int) # Start with zeros
    for k_in in range(state_dim_2w):
        cA_in, pA_in, cB_in, pB_in = get_2walker_coords_from_index(k_in, n_sites_1d)
        if pA_in == pB_in and INTERACTION_ENABLED:
            # Interaction occurs
            p_common = pA_in
            # Map (cA_in, cB_in) to 0..3 index for the 4x4 interaction_gate_2q
            # Assuming interaction_gate_2q basis is |00>_AB, |01>_AB, |10>_AB, |11>_AB (A=LSB for index)
            idx_2q_in = cA_in + 2 * cB_in

            for idx_2q_out in range(4): # Iterate over possible output 2-qbit coin states
                amp_int = interaction_gate_2q[idx_2q_out, idx_2q_in]
                if np.abs(amp_int) > 1e-9:
                    cA_out = idx_2q_out % 2
                    cB_out = idx_2q_out // 2
                    k_out = get_2walker_index(cA_out, p_common, cB_out, p_common, n_sites_1d)
                    U_interaction_final[k_out, k_in] = amp_int
        else:
            # No interaction, walker passes through (Identity)
            U_interaction_final[k_in, k_in] = 1.0

    return U_interaction_final


# --- Observables (2 Walkers - from your original 2-walker code) ---
def calculate_p_pos_2walkers(joint_sv, n_sites_1d, walker_id='A'):
    # ... (same as your original 2-walker code)
    d_1w = 2 * n_sites_1d; d_tot = d_1w * d_1w
    probs_1d = np.zeros(n_sites_1d)
    if len(joint_sv) != d_tot: return probs_1d
    for k_joint in range(d_tot):
        cA, pA, cB, pB = get_2walker_coords_from_index(k_joint, n_sites_1d)
        amp_sq = np.abs(joint_sv[k_joint])**2
        if walker_id == 'A': probs_1d[pA] += amp_sq
        elif walker_id == 'B': probs_1d[pB] += amp_sq
    return probs_1d

def calculate_coin_entanglement_2walkers(joint_sv, n_sites_1d, walker_id='A'):
    # ... (same as your original 2-walker code for S(A_coin) or S(B_coin))
    d_1w = 2 * n_sites_1d; d_tot = d_1w * d_1w; coin_dim = 2
    if len(joint_sv) != d_tot: return np.nan
    norm = np.linalg.norm(joint_sv)
    if norm < 1e-9: return 0.0
    current_psi = joint_sv / norm if np.abs(norm - 1.0) > 1e-6 else joint_sv
    rho_full = np.outer(current_psi, np.conjugate(current_psi))
    rho_coin_walker = np.zeros((coin_dim, coin_dim), dtype=np.complex128)
    for k_joint_row in range(d_tot):
        cA_r, pA_r, cB_r, pB_r = get_2walker_coords_from_index(k_joint_row, n_sites_1d)
        for k_joint_col in range(d_tot):
            cA_c, pA_c, cB_c, pB_c = get_2walker_coords_from_index(k_joint_col, n_sites_1d)
            if walker_id == 'A':
                if pA_r == pA_c and cB_r == cB_c and pB_r == pB_c:
                    rho_coin_walker[cA_r, cA_c] += rho_full[k_joint_row, k_joint_col]
            elif walker_id == 'B':
                if cB_r == cB_c and cA_r == cA_c and pA_r == pA_c:
                     rho_coin_walker[cB_r, cB_c] += rho_full[k_joint_row, k_joint_col]
    tr_rho_coin = np.trace(rho_coin_walker)
    if abs(tr_rho_coin) < 1e-9: return 0.0
    if np.abs(tr_rho_coin - 1.0) > 1e-6 : rho_coin_walker /= tr_rho_coin
    eigs = np.linalg.eigvalsh(rho_coin_walker); ent = 0.0; epsilon = 1e-12
    for e_val in eigs: ent -= (e_val * np.log2(e_val) if e_val > epsilon else 0.0)
    return max(0.0, np.real(ent))

# --- Simulation Loop for 2 Interacting Walkers on Static Substrate ---
def run_2walkers_interacting_on_substrate(
    n_sites_1d, depth,
    pA_init, cA_vec, pB_init, cB_vec,
    static_substrate, c_sub_0, c_sub_1,
    interaction_gate, substrate_name="Substrate"
):
    current_qw_state = prepare_initial_state_2walkers_flexible(n_sites_1d, pA_init, cA_vec, pB_init, cB_vec)

    prob_A_hist = np.full((depth + 1, n_sites_1d), np.nan)
    prob_B_hist = np.full((depth + 1, n_sites_1d), np.nan)
    ent_A_hist = np.full(depth + 1, np.nan) # Ent of coin A with rest
    ent_B_hist = np.full(depth + 1, np.nan) # Ent of coin B with rest
    # Could also add S(A_coin : B_coin) or S(A_walker : B_walker)

    prob_A_hist[0, :] = calculate_p_pos_2walkers(current_qw_state, n_sites_1d, 'A')
    prob_B_hist[0, :] = calculate_p_pos_2walkers(current_qw_state, n_sites_1d, 'B')
    ent_A_hist[0] = calculate_coin_entanglement_2walkers(current_qw_state, n_sites_1d, 'A')
    ent_B_hist[0] = calculate_coin_entanglement_2walkers(current_qw_state, n_sites_1d, 'B')

    print(f"\nStarting 2-Walker (Interacting) QW on {substrate_name} for {depth} steps...")
    interaction_status = "ON" if INTERACTION_ENABLED else "OFF"
    print(f"Interaction Gate between walkers at same site: {interaction_status}")

    start_time = time.time()

    # Precompute operators
    U_Coin_A_op, U_Coin_B_op = build_2walker_coin_operators_on_substrate(n_sites_1d, static_substrate, c_sub_0, c_sub_1)
    S_A_op, S_B_op = build_2walker_shift_operators(n_sites_1d)
    U_Interaction_op = build_2walker_interaction_operator(n_sites_1d, interaction_gate)

    # Step operator: Coins -> Shifts -> Interaction (Interaction can be placed differently)
    # Order based on common QW definitions: U_int @ S @ C
    # Here: U_int @ S_B @ S_A @ C_B @ C_A
    U_step_const = U_Interaction_op @ S_B_op @ S_A_op @ U_Coin_B_op @ U_Coin_A_op

    for step in range(depth):
        current_qw_state = U_step_const @ current_qw_state
        norm_qw = np.linalg.norm(current_qw_state)
        if abs(norm_qw) > 1e-9: current_qw_state /= norm_qw
        else: print(f"Warning: 2W QW Norm zero at step {step+1}."); break

        prob_A_hist[step + 1, :] = calculate_p_pos_2walkers(current_qw_state, n_sites_1d, 'A')
        prob_B_hist[step + 1, :] = calculate_p_pos_2walkers(current_qw_state, n_sites_1d, 'B')
        ent_A_hist[step+1] = calculate_coin_entanglement_2walkers(current_qw_state, n_sites_1d, 'A')
        ent_B_hist[step+1] = calculate_coin_entanglement_2walkers(current_qw_state, n_sites_1d, 'B')

        if (step + 1) % (max(1, depth // 10)) == 0 or step == depth -1:
            print(f"  2W-{substrate_name} Step {step + 1}/{depth}. S(A_c)={ent_A_hist[step+1]:.3f}, S(B_c)={ent_B_hist[step+1]:.3f}")

    end_time = time.time()
    print(f"2-Walker (Interacting) QW on {substrate_name} complete. Time: {end_time - start_time:.2f} seconds.")
    return {
        "prob_A_hist": prob_A_hist, "prob_B_hist": prob_B_hist,
        "ent_A_hist": ent_A_hist, "ent_B_hist": ent_B_hist,
        "static_substrate": static_substrate, # Store it once
        "params": { "n_sites":n_sites_1d,"depth":depth, "substrate_type": substrate_name,
                    "interaction": interaction_status, "interaction_gate_str": str(interaction_gate)}
    }

# --- Plotting for 2 Interacting Walkers (similar to your original 2-walker plot) ---
def plot_2_interacting_walkers_results(results, save_path=None, plot_title_prefix="2 Interacting QWs"):
    params = results["params"]
    pA, pB = results["prob_A_hist"], results["prob_B_hist"]
    eA, eB = results["ent_A_hist"], results["ent_B_hist"]
    static_substrate = results["static_substrate"]
    n_sites = params["n_sites"]; depth = params["depth"]
    substrate_info = params.get("substrate_type", "Substrate")
    interaction_info = params.get("interaction", "N/A")

    fig, axs = plt.subplots(5, 1, figsize=(12, 20), gridspec_kw={'height_ratios': [1.0, 3, 3, 2, 2]})
    title_str = f"{plot_title_prefix} on {substrate_info} (N={n_sites}, D={depth}, Interact: {interaction_info})"
    fig.suptitle(title_str, fontsize=14)

    axs[0].imshow(static_substrate.reshape(1, -1), cmap='binary', aspect='auto', interpolation='nearest', extent=[0, n_sites-1, 0, 1])
    axs[0].set_title(f"Static {substrate_info} Pattern"); axs[0].set_xlabel("Site Index"); axs[0].set_yticks([])

    time_extent = [0, depth, 0, n_sites-1]; norm_v = None # Linear prob scale

    def plot_prob_dist(ax, prob_hist_walker, walker_label, cmap_choice):
        prob_st=prob_hist_walker.T
        im=ax.imshow(prob_st,aspect='auto',origin='lower',cmap=cmap_choice,norm=norm_v,extent=time_extent)
        plt.colorbar(im,ax=ax,label=f"P_{walker_label}(x,t)");ax.set_title(f"Walker {walker_label} P(x,t)");ax.set_xlabel("Time Step");ax.set_ylabel("Site (x)")
    plot_prob_dist(axs[1], pA, 'A', 'viridis')
    plot_prob_dist(axs[2], pB, 'B', 'magma')

    ts=np.arange(depth+1)
    axs[3].plot(ts,eA,marker='.',ls='-',color='blue',label='S(Coin A)'); axs[3].plot(ts,eB,marker='.',ls='-',color='red',label='S(Coin B)')
    axs[3].set_title("Individual Coin Entanglement S(t)"); axs[3].set_xlabel("Time Step"); axs[3].set_ylabel("Entropy S"); axs[3].grid(True); axs[3].legend();axs[3].set_ylim(bottom=-0.05,top=1.05)

    avg_pos_A = np.array([np.sum(pA[t,:] * np.arange(n_sites)) / np.sum(pA[t,:]) if np.sum(pA[t,:]) > 1e-9 else np.nan for t in range(depth+1)])
    avg_pos_B = np.array([np.sum(pB[t,:] * np.arange(n_sites)) / np.sum(pB[t,:]) if np.sum(pB[t,:]) > 1e-9 else np.nan for t in range(depth+1)])
    axs[4].plot(ts, avg_pos_A, marker='.', ls='-', color='green', label='<x_A>')
    axs[4].plot(ts, avg_pos_B, marker='.', ls='--', color='purple', label='<x_B>')
    axs[4].set_title("Average Positions |<x_A> & <x_B>|(t)"); axs[4].set_xlabel("Time Step"); axs[4].set_ylabel("Avg. Position"); axs[4].grid(True); axs[4].legend()

    plt.tight_layout(rect=[0,0,1,0.96])
    if save_path: plt.savefig(save_path); print(f"Plot saved to {save_path}"); plt.close(fig)
    else: plt.show()


# --- Main Execution ---
if __name__ == "__main__":
    output_base_dir = "qw_fibonacci_2walker_interacting"
    if not os.path.exists(output_base_dir): os.makedirs(output_base_dir)

    substrate = generate_fibonacci_sequence_01(N_SITES_1D)

    # --- Run with NO interaction first ---
    print("\n===== 2-Walker Fibonacci QW - NO INTERACTION =====")
    INTERACTION_GATE_2Q_CURRENT = NO_INTERACTION_2Q
    INTERACTION_ENABLED = False # Manually set for clarity
    results_no_int = run_2walkers_interacting_on_substrate(
        n_sites_1d=N_SITES_1D, depth=DEPTH,
        pA_init=INITIAL_POS_A, cA_vec=INITIAL_COIN_A_VEC,
        pB_init=INITIAL_POS_B, cB_vec=INITIAL_COIN_B_VEC,
        static_substrate=substrate, c_sub_0=COIN_SUBSTRATE_0, c_sub_1=COIN_SUBSTRATE_1,
        interaction_gate=INTERACTION_GATE_2Q_CURRENT, substrate_name="Fibonacci"
    )
    plot_2_interacting_walkers_results(results_no_int,
        save_path=os.path.join(output_base_dir, f"Fib_2W_NoInteract_N{N_SITES_1D}_D{DEPTH}.png"))

    # --- Run with CNOT interaction ---
    print("\n===== 2-Walker Fibonacci QW - CNOT INTERACTION =====")
    INTERACTION_GATE_2Q_CURRENT = CNOT_2Q
    INTERACTION_ENABLED = True # Manually set for clarity
    results_cnot_int = run_2walkers_interacting_on_substrate(
        n_sites_1d=N_SITES_1D, depth=DEPTH,
        pA_init=INITIAL_POS_A, cA_vec=INITIAL_COIN_A_VEC, # Same initial conditions
        pB_init=INITIAL_POS_B, cB_vec=INITIAL_COIN_B_VEC,
        static_substrate=substrate, c_sub_0=COIN_SUBSTRATE_0, c_sub_1=COIN_SUBSTRATE_1,
        interaction_gate=INTERACTION_GATE_2Q_CURRENT, substrate_name="Fibonacci"
    )
    plot_2_interacting_walkers_results(results_cnot_int,
        save_path=os.path.join(output_base_dir, f"Fib_2W_CNOT_Interact_N{N_SITES_1D}_D{DEPTH}.png"))


    print("\n<<<<< 2-WALKER INTERACTING QW SIMULATIONS COMPLETE >>>>>")


===== 2-Walker Fibonacci QW - NO INTERACTION =====
Initialized 2-Walker state. A at 10 (coin ~[1.+0.j 0.+0.j]), B at 20 (coin ~[1.+0.j 0.+0.j])

Starting 2-Walker (Interacting) QW on Fibonacci for 50 steps...
Interaction Gate between walkers at same site: OFF
  2W-Fibonacci Step 5/50. S(A_c)=0.573, S(B_c)=0.469
  2W-Fibonacci Step 10/50. S(A_c)=0.777, S(B_c)=0.722
  2W-Fibonacci Step 15/50. S(A_c)=0.951, S(B_c)=0.401
  2W-Fibonacci Step 20/50. S(A_c)=0.810, S(B_c)=0.926
  2W-Fibonacci Step 25/50. S(A_c)=0.798, S(B_c)=0.956
  2W-Fibonacci Step 30/50. S(A_c)=0.959, S(B_c)=0.815
  2W-Fibonacci Step 35/50. S(A_c)=0.826, S(B_c)=0.991
  2W-Fibonacci Step 40/50. S(A_c)=0.809, S(B_c)=0.002
  2W-Fibonacci Step 45/50. S(A_c)=0.959, S(B_c)=0.916
  2W-Fibonacci Step 50/50. S(A_c)=0.901, S(B_c)=0.011
2-Walker (Interacting) QW on Fibonacci complete. Time: 986.63 seconds.
Plot saved to qw_fibonacci_2walker_interacting/Fib_2W_NoInteract_N30_D50.png

===== 2-Walker Fibonacci QW - CNOT INTERACTION ====